### Поведение пользователей приложения. Аналитический отчет.

КЕЙС:

Отделу маркетинга в компании-сервисе по доставке продуктов на дом необходим отчет о поведении пользователей и эффективности каналов привлечения.

ЗАДАЧИ:

- Изучить динамику поведения пользователей в приложении. Выявить зависимости, точки роста. Отследить кол-во инсталлов и первых открытий приложения, их долю в трафике.
- Проанализировать эффективность каналов привлечения по нескольким параметрам: количество инсталлов, конверсия в первую покупку, медианный чек, ROMI, доля в общей прибыли.
- Построить воронку конверсии, выявить слабые точки.
- Провести когортный анализ по нескольким условиям: по неделе первого посещения, по неделе регистрации, по неделе первой покупки.

ОПИСАНИЕ ДАННЫХ:

- Данные выгружены из AppMetrica за период с 1 января по 31 марта 2020 — только по пользователям, зарегистрированным позднее 1 января 2020.
- В выгрузке только уникальные действия пользователей за каждый день.
- Можно миновать стадию установки приложения, если оно было установлено ранее.
- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку.


### Исследование и подготовка данных ###

In [1]:
import pandas as pd #импортируем все необходимые модули и библиотеки
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import timedelta


In [2]:
df = pd.read_csv(r'C:\Users\admin\Documents\IDE\Data\SF_case_data.csv') #читаем исходный файл SF_case_data

In [3]:
df.head() #изучаем данные

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [4]:
df.info() #Изучаем типы данных в столбцах

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747968 entries, 0 to 2747967
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   date          object 
 1   event         object 
 2   purchase_sum  float64
 3   os_name       object 
 4   device_id     int64  
 5   gender        object 
 6   city          object 
 7   utm_source    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 167.7+ MB


In [5]:
df['date'] = pd.to_datetime(df['date']) #меняем тип данных на datetime
df.loc[df['utm_source']=='-','utm_source']='unknown/direct' #переименовываем пустое значение в столбце utm_source

In [6]:
print(df['event'].unique()) #изучим варианты значений в столбцах event и utm_source
print(df['utm_source'].unique())

['app_start' 'choose_item' 'purchase' 'search' 'tap_basket' 'app_install'
 'register']
['unknown/direct' 'vk_ads' 'referal' 'facebook_ads' 'google_ads'
 'instagram_ads' 'yandex-direct']


In [7]:
na_df = df.isna() #с помощью na_df проверим кол-во пропусков. Пропуски есть в только столбце purchase_sum - это логично
na_df.value_counts()

date   event  purchase_sum  os_name  device_id  gender  city   utm_source
False  False  True          False    False      False   False  False         2606585
              False         False    False      False   False  False          141383
dtype: int64

In [8]:
users_with_min_dates = df.pivot_table(index='device_id',columns='event',values='date',aggfunc='min').reset_index()
users_with_min_dates # создадим таблицу, где зафиксируем даты первых событий для каждого пользователя

event,device_id,app_install,app_start,choose_item,purchase,register,search,tap_basket
0,4013,2020-01-15,2020-01-15,2020-01-15,NaT,NaT,2020-01-15,2020-01-15
1,4014,NaT,2020-01-02,2020-01-02,2020-02-01,2020-02-01,2020-01-02,2020-01-02
2,4016,2020-01-04,2020-01-04,2020-01-04,NaT,NaT,2020-01-04,2020-02-15
3,4018,NaT,2020-03-27,2020-03-27,NaT,NaT,2020-03-27,2020-03-27
4,4046,2020-01-04,2020-01-04,2020-01-04,2020-01-04,2020-01-04,2020-01-04,2020-01-04
...,...,...,...,...,...,...,...,...
190879,35379281,2020-03-29,2020-03-29,NaT,NaT,NaT,2020-03-29,NaT
190880,35380796,2020-03-31,2020-03-31,2020-03-31,NaT,NaT,2020-03-31,2020-03-31
190881,35381595,2020-03-30,2020-03-30,2020-03-30,NaT,2020-03-30,2020-03-30,2020-03-30
190882,35388218,2020-03-31,2020-03-31,2020-03-31,NaT,NaT,2020-03-31,NaT


### Динамика событий ###

In [11]:
df_grouped_dinamics = df.groupby(by='date',as_index=False)['event'].count() #сгруппируем данные по датам, посчитаем кол-во событий

In [12]:
df['day_of_week'] = df['date'].dt.day_of_week #Выделим у каждого события новый признак "день недели"
df_grouped_dinamics_dow = df.groupby(by='day_of_week',as_index=False)['event'].count() # Сгруппируем по дням недели,
# посчитаем кол-во событий

In [13]:
fig = px.line(
    data_frame=df_grouped_dinamics_dow, 
    x='day_of_week', 
    y='event', 
    height=500, 
    width=1000, 
    title='Динамика событий по дням недели',
    line_shape = 'spline'
)
fig.show()

In [14]:
fig = px.line(
    data_frame=df_grouped_dinamics, 
    x='date', 
    y='event', 
    height=500,
    width=1000, 
    title='Динамика событий',
    line_shape = 'spline'
)
fig.show()

### Выводы и рекомендации ###

1. Динамика событий в приложении очень неравномерна. Имеются точки резкого роста, которые сменяются таким же резким спадом. Скорее всего это можно объяснить каледарными событиями, когда пользователи покупают больше (10 января - возвращение домой после каникул, 14 февраля/21 февраля/6 марта - закупки накануне праздников).
2. Есть сильная зависимость от дня недели. Активность очень высока в пятницу, снижается в субботу и воскресенье. В остальные дни недели - на очень низком уровне.
3. Есть возможность роста за счет увеличения продаж именно в периоды спада. Рекламные кампании в будние дни и дни, следующие за большими праздниками.

### Динамика открытий приложения для групп "Установили в тот же день" и "Остальные" ###

In [15]:
#создадим новый df, к которому добавим столбец с датой app_install для каждого пользователя
df_dinamics_app_start = df
df_dinamics_app_start  = df_dinamics_app_start.merge (users_with_min_dates,on='device_id',how='left')
df_dinamics_app_start = df_dinamics_app_start.drop(['choose_item','search','tap_basket','purchase','app_start','register'],axis=1)

In [16]:
#оставим только события app_start и удалим лишние столбцы
df_dinamics_app_start = df_dinamics_app_start[df_dinamics_app_start['event']=='app_start']
df_dinamics_app_start = df_dinamics_app_start[['date','device_id','app_install']]

In [17]:
#Создадим функцию для определения категории 
def install_appstart_category(row):
    if row['app_install']==row['date']:
        return 'установили в тот же день'
    else:
        return 'остальные'   

In [18]:
#Применим функцию и получим новый столбец с категорией
df_dinamics_app_start['install_appstart_category'] = df_dinamics_app_start.apply(install_appstart_category,axis=1)

In [19]:
#Сгруппируем данные по дате и посчитаем кол-во пользователей по категориям
df_dinamics_app_start = df_dinamics_app_start.pivot_table(values='device_id',aggfunc='count',index='date',columns='install_appstart_category').reset_index()

In [20]:
#Добавим столбец 'total' с общим кол-ом пользователей
df_dinamics_app_start['всего открытий'] = df_dinamics_app_start['установили в тот же день'] + df_dinamics_app_start['остальные']

In [21]:
fig = px.line(
    data_frame=df_dinamics_app_start, 
    x='date', 
    y=['установили в тот же день','остальные','всего открытий'], 
    height=500, 
    width=1000, 
    title='Динамика открытий приложения',
    line_shape = 'spline'
)
fig.show()

### Выводы ###

1. Сохраняется общая динамика для обеих групп ("установили в тот же день" и "остальные"). Рост и падения собпадают. Исключения - 1,2 января (что можно объяснить низкой активностью категории "остальные" в связи с праздником) и 10 января, когда активность новых пользователей превышала категорию "остальные".
2. Можно с достаточной уверенностью утверждать, что первый день использования приложения, первый опыт взаимодействия - очень важен. Необходимо стремится к улучшению первого впечатления, используя опыт 10 января 2020.

### Доля инсталлов в трафике ###

In [22]:
#отфильтруем только событий 'app_install', сгруппируем по дате, посчитаем кол-во событий
df_installs_traffic = df[df['event']=='app_install']
df_installs_traffic = df_installs_traffic.groupby('date',as_index=False)['event'].count()

In [23]:
#присоединим df_installs_traffic с общим кол-ом событий по дате, переименуем столбцы, создадим столбец installs_proportion - доля 
# инсталлов с трафике
df_installs_traffic = df_installs_traffic.merge(df_grouped_dinamics,how='left',on='date')
df_installs_traffic = df_installs_traffic.rename(columns={'event_x':'install_count','event_y':'total_event_count'})
df_installs_traffic['installs_proportion'] = round(df_installs_traffic['install_count']/df_installs_traffic['total_event_count']*100)

In [24]:
fig = px.line(
    data_frame=df_installs_traffic, 
    x='date', 
    y=['installs_proportion'], 
    height=500, 
    width=1000, 
    title='Доля инсталлов от общего трафика в %',
    line_shape = 'spline'
)
fig.show()

### Выводы ###

Доля инсталлов в трафике - это важный показатель, характеризующий приток новых пользователей в приложение. Самые высокие и самые низкие показатели по конкретным дням необходимо изучать и корректировать маркетинговые ходы.

### Установки приложения по каналам привлечения ###

In [25]:
#создадим список пользователей, которые установили приложение в исследуемом периоде
users_with_install_list = df[df['event']=='app_install']['device_id'].to_list()

In [26]:
#отфильтурем данные, оставив только действия пол-ей, которые установили приложение в исследуемом периоде
#добавим столбец с датой app_install для каждого пользователя
users_with_install = df[df['device_id'].isin(users_with_install_list)]
users_with_install  = users_with_install.merge (users_with_min_dates,on='device_id',how='left')
users_with_install = users_with_install.drop(['choose_item','search','tap_basket','purchase','app_start','register'],axis=1)

In [27]:
#создадим функцию для определения категории "первое открытие приложения"
def first_app_start_category(row):
    if row['event']=='app_start' and row['date']==row['app_install']:
        return 'yes'
    else:
        return 'no'       

In [28]:
#применим функцию и создадим новый столбец с категорией
users_with_install['category_first_app_start'] = users_with_install.apply(first_app_start_category,axis=1)

In [29]:
#отфильтруем только события "первого открытия", сгруппируем по каналам, посчитаем пользователей
users_grouped_first_app_start = users_with_install[users_with_install['category_first_app_start']=='yes']
users_grouped_first_app_start = users_grouped_first_app_start.groupby('utm_source',as_index=False)['device_id'].count()

In [30]:
#отфильтруем только события 'app_install', сгруппируем по каналам, посчитаем пользователей
users_grouped_install = df[df['event']=='app_install']
users_grouped_install = users_grouped_install.groupby('utm_source',as_index=False)['device_id'].count()

In [31]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],subplot_titles=['Инсталл', 'Первое открытие приложения'])

fig.add_trace(go.Pie(labels=users_grouped_install['utm_source'], values=users_grouped_install['device_id']),
              1, 1)              
fig.add_trace(go.Pie(labels=users_grouped_first_app_start['utm_source'], values=users_grouped_first_app_start['device_id']),
              1, 2)


fig.update_layout(title_text="Привлечение пользователей по каналам")
fig.show()

### Выводы ###
1. Данные по инсталлам и первым открытиям приложения в разрезе каналов привлечения совпадают.
   
2. Оставляя за скобками канал 'unknown/direct' - когда мы не можем определить канал привлечения, можно отметить, что больше всего новых пользователей приходит с канал ЯндексДирект - 19%, с недольшим отрывом следуют Гугл - 17% и Вконтакте - 15%. Меньше всего пользователей приносит реферальная программа - 9%.

### Воронка конверсии ###

In [32]:
#создадим users_regist_count, добавим столбец с датой регистрации 'register'
users_regist_count = df
users_regist_count = users_regist_count.merge (users_with_min_dates,on='device_id',how='left')
users_regist_count = users_regist_count.drop(['choose_item','search','tap_basket','purchase','app_start','app_install'],axis=1)

In [33]:
#Создадим функцию для определения категории события: зарегистрирован пользователь на момент события или нет
def regist_type (row):
    if row['register']<row['date']:
        return 'зарегистрирован'
    if row['register']>row['date'] or row['register']==row['date']:
        return 'незарегистрирован'
    else:
        return 'незарегистрирован'

In [34]:
#Применим функцию, создадим новый столбец с категорией
users_regist_count['regist_type'] = users_regist_count.apply(regist_type,axis=1)

In [35]:
users_regist_count = users_regist_count[users_regist_count['event']!='app_install'] #отфильтруем событие app_install, потому что многие
#пользователи загрузили приложение ранее (не в исследуемом квартале), данные не важны для анализа воронки
users_regist_count= users_regist_count.pivot_table(index='event',columns='regist_type',values='date',aggfunc='count').sort_values(by='незарегистрирован',ascending=False).reset_index()
 #сгруппируем данные по событиям, посчитаем события с каждой категории

In [36]:
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'незарегистрирован',
    y = users_regist_count['event'],
    x = users_regist_count['незарегистрирован'],
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name = 'зарегистрирован',
    orientation = "h",
    y = users_regist_count['event'],
    x = users_regist_count['зарегистрирован'],
    textposition = "inside",
    textinfo = "value+percent previous"))

fig.update_layout(title_text="Воронка конверсии")

fig.show()

### Выводы и рекомендации ###

1. На всех этапах воронки количество зарегистрированных пользователей значительно превышает незарегистрированных. Следовательно регистрация - очень важный этап, на котором важно концентрировать усилия маркетинга.
2. Относительные показатели перехода на следующий этап воронки очень близки для категорий до этапа Регистрация или Покупка.
3. Самая слабая точка для зарегистрированных пользователей - переход от корзины к покупке (только 34%).
4. Самая слабая точка для незарегистрированных пользователей - переход от корзины к регистрации (только 49%). Далее идет очень уверенная конверсия в покупку - 87%. Т.е. основная рекомендация - упрощение, ускорение этапа регистрации; дополнительное мотивирование пользователя на этом этапе.

### Конверсия в первую покупку по каналам привлечения ###

In [37]:
#Создадим dataframe, в которую добавим столбец с датой первой покупки для каждого пользователя
df_first_purchase = df
df_first_purchase = df_first_purchase.merge (users_with_min_dates,on='device_id',how='left')
df_first_purchase = df_first_purchase.drop(['choose_item','search','tap_basket','register','app_start','app_install'],axis=1)

In [38]:
#Функция для определения категории "Первая покупка"
def first_purchase_type(row):
    if row['event']=='purchase' and row['date']==row['purchase']:
        return 'first_purchase_type'
    else:
        return 'other'

In [39]:
#Применим функцию, создадим новый столбец, отфильтруем данные, оставив только первые покупки
df_first_purchase['first_purchase_type'] = df_first_purchase.apply(first_purchase_type,axis=1)
df_first_purchase = df_first_purchase[df_first_purchase['first_purchase_type']=='first_purchase_type']

In [40]:
#Сгруппируем данные по каналам, посчитаем кол-во пользователей, совершивших первую покупку.
#Присоединим созданную ранее users_grouped_first_app_start, те кол-во пользователей, совершивших первое открытие приложения
#Переименуем столбцы для удобного использования
df_CR_first_purchase = df_first_purchase.groupby(by='utm_source',as_index=False)['device_id'].count()
df_CR_first_purchase = df_CR_first_purchase.merge(users_grouped_first_app_start, on='utm_source',how='left')
df_CR_first_purchase = df_CR_first_purchase.rename(columns={'device_id_x':'first_purchase_count','device_id_y':'first_appstart_count'})

In [41]:
#Создадим новый столбец - Конверсия в первую покупку (CR = количество пользователей, впервые совершивших 
# покупку/количество пользователей, впервые открывших приложение.) Отсортируем данные.
df_CR_first_purchase['first_purchase_CR'] = round(df_CR_first_purchase['first_purchase_count']/df_CR_first_purchase['first_appstart_count']*100)
df_CR_first_purchase = df_CR_first_purchase.sort_values(by='first_purchase_CR',ascending=False)

In [42]:
fig = px.bar(
    data_frame=df_CR_first_purchase, 
    x="utm_source", 
    y="first_purchase_CR", 
    color='utm_source', 
    text = 'first_purchase_CR', 
    height=500, 
    width=1000, 
    title='Конверсия в первую покупку по каналам в %'
)
fig.show()

### Выводы ###

1. Реферальная программа  - лучший канал привлечения по конверсии в первую покупку - 63% (это объяснимо условиями программы - участники получают вознаграждение только при условии успешной первой покупки). 
2. Остальные каналы показывают чуть менее высокие результаты. Самые низкие показатели у Гугл и Яндекс - 39% и 37% соответственно.

### Медианный чек первой покупки по каналам ###

In [43]:
# Используем созданную ранее df_first_purchase, где отображены данные по первым покупкам
# Сгруппируем данные по каналам, посчитаем медианный чек, отсортируем данные
Check_first_purchase = df_first_purchase.groupby(by='utm_source',as_index=False)['purchase_sum'].median().sort_values(by='purchase_sum',ascending=False)

In [44]:
fig = px.bar(
    data_frame=Check_first_purchase, 
    x="utm_source", 
    y="purchase_sum", 
    color='utm_source', 
    text = 'purchase_sum', 
    orientation='v', 
    height=500, 
    width=1000, 
    title='Медианный чек первой покупки' 
)
fig.show()

### Выводы ###

Канал привлечения влияет на сумму медианного чека очень незначительно. Показатели отличаются друг от друга в пределах 10 рублей.

### ROMI по каналам ###

In [45]:
#Создадим DataFrame с данными о затратах на рекламу (взяты из учебного модуля)
marketing_costs_df = pd.DataFrame(
    data=[
        ['yandex-direct',10491707],
        ['google_ads',10534878],
        ['facebook_ads',8590498],
        ['instagram_ads',8561626],
        ['vk_ads',9553531]
        ],
    columns=['utm_source','marketing_costs'],
    )

In [46]:
#Отфильтруем все первые покупки с канала "referal", сосчитаем их кол-во и умножим на 200 (условие задания "Расходы на реферальную
# программу: если пользователь приведет друга и последний совершит первую покупку, то оба получат по 100 рублей.")
#В результате получаем данные о маркетинговых затратах на реферальную программу
referal_costs = df_first_purchase[df_first_purchase['utm_source']=='referal']
referal_costs = referal_costs.groupby(by='utm_source',as_index=False)['event'].count()
referal_costs['marketing_costs'] = referal_costs['event']*200
referal_costs = referal_costs.drop (columns='event')

In [47]:
#Объединяем данные построчно, получаем новую dataframe с маркетинговыми расходами для всех каналов
romi_df = pd.concat([marketing_costs_df,referal_costs],ignore_index=True)

In [48]:
#Посчитаем выручку по каналам
revenue_df = df.groupby(by='utm_source',as_index=False)['purchase_sum'].sum()
revenue_df = revenue_df.rename(columns={'purchase_sum':'revenue'})

In [49]:
#Объеденим данные по выручке и затратах, расчитаем новый столбец ROMI, отсортируем данные
romi_df = romi_df.merge(revenue_df,on='utm_source',how='left')
romi_df['ROMI_%'] = round((romi_df['revenue']-romi_df['marketing_costs'])/romi_df['marketing_costs']*100)
romi_df = romi_df.sort_values(by='ROMI_%',ascending=False)

In [50]:
fig = px.histogram(romi_df, x="utm_source", y=['ROMI_%'],
             color='utm_source',
             width=1000,
             text_auto=True,
             title='ROMI по каналам в %')
            
fig.show()

### Выводы ###

1. Показатель ROMI расчитывался как (Выручка - Расходы) / Расходы * 100%. По данному показателю Реферальная программа с большим отрывом опережает остальные каналы - 661%. Самые низкие показатели у Яндекс и Гугл - 33% и 22% соответственно. 

### Суммарная прибыль по каналам ###

In [51]:
df_income_by_source = df.pivot_table (index='utm_source',values='purchase_sum',aggfunc='sum').reset_index()
#Сгруппируем данные по каналам, суммируем покупки

In [52]:
fig = px.pie(df_income_by_source, values='purchase_sum', names='utm_source', title='Суммарная прибыль по каналам')
fig.show()

### Вывод ###

Все каналы привлечения имеют ощутимую долю в общей прибыли. Лидеры - ВК, Инстаграм.

### Когорты по неделе первого посещения ###

In [53]:
appstarts = df[df['event'] == 'app_start'].sort_values('date').drop_duplicates('device_id') #Отфильтруем только события app_start,
# сортируем по дате, удаляем повторения 'device_id'
df['first_entrance'] = df['device_id'].map(appstarts.set_index('device_id')['date']) #Подтягиваем данные из appstarts,
#Получая таким образом даты первых открытий приложения для каждого пользователя

In [54]:
df['first_entrance_cohort'] = df['first_entrance'].apply(lambda x: x + timedelta(days=-x.weekday(), weeks=0))# Создаем столбец,
#В который с помощью функции попадает первый день недели для каждого события (название когорты)
df['n_week'] = df['date'] - df['first_entrance_cohort'] #Вычисляем разницу между событием с когортой в днях
df['n_week'] = df['n_week'].apply(lambda x: x.days // 7 ) #Переводим дни в недели
df['first_entrance_cohort'] = df['first_entrance_cohort'].apply(lambda x: str(x)[:10]) #переводим столбец в формат строки

In [55]:
#Создаем таблицу когортного анализа группируя данные по когортам, рассматриваем кол-во уникальных пользователей по неделям
viz = df.pivot_table(
                        index='first_entrance_cohort', 
                        columns='n_week', 
                        values='device_id', 
                        aggfunc=pd.Series.nunique)

In [56]:
fig = px.imshow(viz, text_auto=True,color_continuous_scale = 'YlGnBu', title='Когорты по первому посещению, активность')
fig.show()

In [57]:
viz = viz.apply(lambda x: round(x*100/viz[0]))
fig = px.imshow(viz, text_auto=True,color_continuous_scale = 'YlGnBu', title='Когорты по первому посещению, активность в %')
fig.show()

### Выводы ###

1. Когортный анализ проводился по неделе первого посещения приложения. Активность когорт расчитывалась как кол-во уникальных пользователей пользующихся приложением.
2. Следует отметить, что 3 первых когорты (3 первые недели января) очень сильно отличаются от общей картины.
3. Самые большие когорты - 6 января 2020 и 13 января 2020 (31 и 30 тысяч новых пользователей) в несколько раз превышают данные по другим когортам. Также они демонстрируют уверенную активность во времени. Например когорта 6 января 2020 показывает активность 4% на 12ой неделе, по сравнению с такой же активностью когорты 23 марта 2020 на 1ой неделе.
4. Однако самой активной когортой является 30 декабря 2019. Несмотря на то,что кол-во новых пользователей в ней среднее(приблизительно 15 тыс), данная когорта очень активна во времени. Очень высокая возвращаемость на 1ой неделе - 48%, и сохраняется очень высокой вплоть до 14ой недели - 14%.
5. Также хочется отметить когорты 24 февраля 2020 и 9 марта 2020 и их отличия от общей картины. Обе когорты сравнительно малы (приблизительно 5 тыс новых пользователей в каждой), но демонтрируют более высокую активность во времени.
6. Видна общая тенденция снижения активности с каждой новой когортой.

### Когорты по неделе регистрации ###

In [58]:
registrations = df[df['event'] == 'register'].sort_values('date').drop_duplicates('device_id') #Отфильтруем только события register,
# сортируем по дате, удаляем повторения 'device_id'
df['first_registration'] = df['device_id'].map(registrations.set_index('device_id')['date']) #Подтягиваем данные из registrations,
#Получая таким образом даты регистрации для каждого пользователя

In [59]:
df_registration = df[-df['first_registration'].isna()] #Отсортируем только пол-ей с регистрацией
df_registration['first_registration_cohort'] = df_registration['first_registration'].apply(lambda x: x + timedelta(days=-x.weekday(), weeks=0))
# Создаем столбец,в который с помощью функции попадает первый день недели для каждого события (название когорты)
df_registration['n_week'] = df_registration['date'] - df_registration['first_registration_cohort'] #Вычисляем разницу между событием с когортой в днях
df_registration['n_week'] = df_registration['n_week'].apply(lambda x: x.days // 7 )  #Переводим дни в недели 


C:\Users\admin\AppData\Local\Temp\ipykernel_14328\2479952315.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\admin\AppData\Local\Temp\ipykernel_14328\2479952315.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\admin\AppData\Local\Temp\ipykernel_14328\2479952315.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [60]:
#Создаем таблицу когортного анализа группируя данные по когортам, рассматриваем кол-во уникальных пользователей по неделям
viz_regist = df_registration.pivot_table(
                        index='first_registration_cohort', 
                        columns='n_week', 
                        values='device_id', 
                        aggfunc=pd.Series.nunique)

In [61]:
fig = px.imshow(viz_regist, text_auto=True,color_continuous_scale = 'YlGnBu', title='Активность когорт по неделе регистрации')
fig.show()

 ### Выводы ###
Данная тепловая карта хорошо демонстрирует,что часть пользователей пользуется приложением некоторое время, а регистрируется уже позднее.

### Когорты по неделе первой покупки ###

In [62]:
purchases = df[df['event'] == 'purchase'].sort_values('date').drop_duplicates('device_id') #Отфильтруем только события purchase,
# сортируем по дате, удаляем повторения 'device_id'
df['first_purchase'] = df['device_id'].map(purchases.set_index('device_id')['date']) #Подтягиваем данные из purchases,
#Получая таким образом даты первых открытий приложения для каждого пользователя

In [63]:
df_purchases = df[-df['first_purchase'].isna()]#Отсортируем только пол-ей с покупкой
df_purchases['first_purchase_cohort'] = df_purchases['first_purchase'].apply(lambda x: x + timedelta(days=-x.weekday(), weeks=0))
# Создаем столбец,в который с помощью функции попадает первый день недели для каждого события (название когорты)
df_purchases['n_week'] =df_purchases['date'] - df_purchases['first_purchase_cohort'] #Вычисляем разницу между событием с когортой в днях
df_purchases['n_week'] = df_purchases['n_week'].apply(lambda x: x.days // 7 ) #Переводим дни в недели 
df_purchases['first_purchase_cohort'] = df_purchases['first_purchase_cohort'].apply(lambda x: str(x)[:10]) #переводим столбец в формат строки

C:\Users\admin\AppData\Local\Temp\ipykernel_14328\2271535173.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\admin\AppData\Local\Temp\ipykernel_14328\2271535173.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\admin\AppData\Local\Temp\ipykernel_14328\2271535173.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [64]:
#Создаем таблицу когортного анализа группируя данные по когортам, рассматриваем кол-во уникальных пользователей по неделям
viz_cohorts_activity = df_purchases.pivot_table(
                        index='first_purchase_cohort', 
                        columns='n_week', 
                        values='device_id', 
                        aggfunc=pd.Series.nunique)

In [65]:
fig = px.imshow(viz_cohorts_activity, text_auto=True,color_continuous_scale = 'YlGnBu',title='Когорты по первой покупке, активность')
fig.show()

In [66]:
viz_cohorts_activity_perc = viz_cohorts_activity.apply(lambda x: round(x*100/viz_cohorts_activity[0]))
fig = px.imshow(viz_cohorts_activity_perc, text_auto=True,color_continuous_scale = 'YlGnBu',title='Когорты по первой покупке, активность в %')
fig.show()

### Выводы ###

1. Когортный анализ проводился по неделе первой покупки. Активность когорт расчитывалась как кол-во уникальных пользователей пользующихся приложением.
2. Основные тенденции совпадают с когортным анализом по первому посещению. Самые большие когорты - 6 января 2020 и 13 января 2020 - 11 и 13 тыс пользователей. Самая активная когорта - 30 декабря 2019, сохраняет активность 33% на 12 неделе существования.
3. Данные по относительной активности во времени постепенно снижаются для каждой новой когорты, но снижение не такое резкое как для когорт по первому посещению. Можно сделать вывод, что пользователи однажды совершившие покупку впоследствие чаще пользуются приложением.

In [67]:
#Создаем таблицу когортного анализа группируя данные по когортам, рассматриваем медианный чек по неделям
viz_median = df_purchases.pivot_table(
                        index='first_purchase_cohort', 
                        columns='n_week', 
                        values= 'purchase_sum', 
                        aggfunc= 'median')

In [68]:
#Создаем таблицу когортного анализа группируя данные по когортам, рассматриваем средний чек по неделям
viz_average = df_purchases.pivot_table(
                        index='first_purchase_cohort', 
                        columns='n_week', 
                        values='purchase_sum', 
                        aggfunc='mean')

In [69]:
fig = px.imshow(viz_median,
                text_auto=True,
                color_continuous_scale = 'YlGnBu',
                title='Медианный чек по когортам')
fig.show()

fig = px.imshow(viz_average,
                text_auto=True,
                color_continuous_scale = 'YlGnBu',
                title='Средний чек по когортам')
fig.show()

### Выводы ###

1. Отметим, что показатели среднего чека превышают медианный приблизительно в 2 раза. Значит часть покупателей совершает покупки на суммы значительно превышающие медианный чек.
2. Сумма медианного чека имеет тенденцию расти для всех когорт. Особенно заметен рост на 1ой, 4ой, 10-11ой неделях. Значит пользователи со временем тратят все больше.
3. Сумма среднего чека колеблется в пределах 600-800 рублей для всех когорт. Не прослеживается явного роста или падения во времени. Видимо очень большие покупки совершаются случайно и не зависят от когорты или возраста когорты.
4. Стоит обратить внимание на высокие показатели среднего чека (превышение почти в раза) для когорт 9 марта 2020 и 16 марта 2020 на 3-ей и 2-ой неделе соответственно.

In [70]:
#Создаем таблицу когортного анализа группируя данные по когортам, рассматриваем суммарную прибыль
viz_purchases_sum = df_purchases.pivot_table(
    index= 'first_purchase_cohort',
    values='purchase_sum',
    aggfunc='sum'
    ).reset_index()

In [71]:
fig = px.bar(
    data_frame=viz_purchases_sum, 
    x="first_purchase_cohort", 
    y="purchase_sum", 
    color='first_purchase_cohort', 
    text = 'purchase_sum',
    text_auto='.2s',
    orientation='v', 
    height=500, 
    width=1000, 
    title='Суммарная прибыль по когортам')
fig.show()

### Выводы ###

1. График суммарной прибыли по когортам лишний раз демонстрирует, насколько эффективно во всех отношениях показывают себя когорты 6 января и 13 января 2020. 
2. Далее прибыль падает с каждой новой когортой. Важно отметить, что показатели когорт конца февраля и марта особенно низкие, так как мы не имеем данных после 31 марта.

In [72]:
#Создаем таблицу когортного анализа группируя данные по каналам привлечения, рассматриваем медианный чек по неделям
viz_utm = df_purchases.pivot_table(
    index= 'utm_source',
    columns='n_week',
    values='purchase_sum',
    aggfunc='median'
    )

In [73]:
fig = px.imshow(viz_utm,
                text_auto=True,
                color_continuous_scale = 'YlGnBu',
                title='Медианный чек по каналам привлечения')
fig.show()

### Выводы ###

1. Не прослеживается явной связи медианного чека от каналов привлечения. Показатели почти равны на 0-ой неделе и имеют тенденцию расти со временем.

### Финальные выводы и рекомендации ###

ПОВЕДЕНИЕ ПОЛЬЗОВАТЕЛЕЙ

- Динамика активности пользователей очень неравномерна. Есть периоды резкого роста (10 января, 14 февраля, 21 февраля,6 марта), которые сменяются периодами резкого падения. Очень выразительна зависимость от дня недели - резкий подъем перед выходными, спад в будние дни.
  
- Рекомендуется планировать дальнейшие маркетинговые кампании исходя из представленного опыта: повторять стратегии на точках роста, разработать решения для периодов падения.
  
- Анализ общей динамики и когортный анализ выявил особенно эффективный период - первые недели января 2020 года. Данный период превышает остальные по множеству показателей: общая активность, количество новых пользователей, количество регистраций, количество первых покупок, доля инсталлов в траффике. Далее когортный анализ показал также очень высокие результаты для когорт: активность, возвращаемость и суммарная прибыль.
  
- Рекомендуется внимательно изучить маркетинговые ходы, использованные в начале января 2020 и проследить связь со столь высокими результатами.
- При изучении воронки конверсии выявлена слабая точка - переход к регистрации для незарегистрированных пользователей, на которой теряется около 50% клиентов. Рекомендуется максимально упростить и ускорить этот шаг.

КАНАЛЫ ПРИВЛЕЧЕНИЯ

- Каналы привлечения были изучены по разным показателям эффективности. Лидерами по количеству новых привлеченных пользователей являются - ЯндексДирект, Гугл, ВК. Самая высокая конверсия в первую покупку - Реферальная программа и Фейсбук. Самый высокий ROMI  - Реферальная программа. Лидеры по доле в общей прибыли - ВК и Инстаграм. 
  
- Рекомендуется продолжать развивать все каналы привлечения. Ни один из каналов нельзя назвать явным аутсайдером.